In [1]:
library(tidyverse)
library(RPostgreSQL)
library(lubridate)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [2]:
drv <- dbDriver("PostgreSQL")

In [3]:
user <- pw <- "datathon"
dbname <- "mimic"

In [4]:
con <- dbConnect(drv, dbname = dbname, user = user, password = pw)

In [5]:
que <- "
    SELECT subject_id, hadm_id, icustay_id FROM (SELECT *,
        MIN(intime) over (PARTITION BY subject_id) AS first
        FROM icustays) AS icu
    WHERE intime = first;
"
quedf <- dbGetQuery(con, que)
head(quedf)

subject_id,hadm_id,icustay_id
2,163353,243653
3,145834,211552
4,185777,294638
5,178980,214757
6,107064,228232
7,118037,278444


In [7]:
data.table::fwrite(quedf, "/Data/common/first_icu_id.csv")